In [1]:
import requests

修改大写部分，一个是TOKEN部分，另一个是labels改为你负责的version

In [16]:
headers = {
    'Authorization': 'token YOUR_TOKEN',
    'Accept': 'application/vnd.github.v3+json'
}

params = {
    "labels": "v1.5",
    "state": "all"
}

api_url = 'https://api.github.com/repos/apache/superset/issues?'
page = 1
while True:
    response = requests.get(api_url + f'&per_page=100&page={page}',params=params,headers=headers)
    issues = response.json()
    if not issues:
        break
    for issue in issues:
        if issue is None:
            continue
        if issue == 'message' or issue == 'documentation_url':
            continue
        if issue['body'] is None:
            continue
        index = issue['number']
        index = str(index)
        labels_info = issue['labels']
        labels = []
        for label_info in labels_info:
            labels.append(label_info['name'])
        created_at = issue['created_at']
        closed_at = issue['closed_at']
        if 'pull_request' in issue:
            pull_request = True
        else:
            pull_request = False
        user_info = issue['user']
        user = user_info['login']
        with open('./data/superset_'+index+'.txt','w') as f:
            f.write('ISSUE_INFO\r\n')
            f.write(str(user))
            f.write('\r\n')
            f.write(str(created_at))
            f.write('\r\n')
            f.write(str(closed_at))
            f.write('\r\n')
            if(pull_request):
                f.write('pull_request')
                f.write('\r\n')
            else:
                f.write('not_pull_request')
                f.write('\r\n')
            for label in labels:
                f.write(label+',')
            f.write('\r\n')
            f.write('BEGIN_ISSUE\r\n')
            f.write(issue['body'])
            f.write('\r\nANOTHER_TEXT_BEGIN\r\n')
            comments_url = issue['comments_url']
            response = requests.get(comments_url,headers=headers)
            comments = response.json()
            for comment in comments:
                if comment is None:
                    continue
                if comment == 'message' or comment == 'documentation_url':
                    continue
                f.write(comment['body'])
                f.write('\r\nANOTHER_TEXT_BEGIN\r\n')
            f.close()
    page = page + 1


ProxyError: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /repos/apache/superset/issues/19547/comments (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))